Neste notebook, consumiremos a api do youtube a fim de adicionar à nossa base de dados a data de publicação como cluna.

Posteriormente, ajustaremos tipos de dados e adicionaremos a categoria `frontline` como valor `booleano`

Importaremos o arquivo pickle pois ele mantém os tipos, tirando redundância de operações dentro de um ambiente `python`. Entretando, nos certificaremos de sempre manter arquivos .csv como referência.

In [220]:
import pandas as pd

weekly_charts = pd.read_pickle("../data/trusted/weekly_charts.pkl")
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7100 non-null   int64         
 1   Previous Rank   7100 non-null   int64         
 2   Track Name      7100 non-null   object        
 3   Artist Names    7100 non-null   object        
 4   Weeks on Chart  7100 non-null   int64         
 5   Views           7100 non-null   int64         
 6   Weekly Growth   7100 non-null   float64       
 7   YouTube URL     7100 non-null   object        
 8   week_ref        7100 non-null   int64         
 9   week_open       7100 non-null   datetime64[ns]
 10  week_close      7100 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 610.3+ KB


O método será filtrar por urls únicas e fazer um request para a api do youtube a partir dos IDs contidos nessas urls. Por fim, adicionaremos ao dataframe por meio de um join.

In [221]:
unique_urls = weekly_charts['YouTube URL'].unique()
df_urls = pd.DataFrame(unique_urls, columns=['YouTube URL'])
df_urls.info()
df_urls.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   YouTube URL  756 non-null    object
dtypes: object(1)
memory usage: 6.0+ KB


,YouTube URL
746,https://www.youtube.com/watch?v=bydRD0taXIY
747,https://www.youtube.com/watch?v=0yPQnBG0hzQ
748,https://www.youtube.com/watch?v=QdWAkRK3HOk
749,https://www.youtube.com/watch?v=HPM7OWuchFs
750,https://www.youtube.com/watch?v=FtHBS6WLsfo
751,https://www.youtube.com/watch?v=6Pix8tMSN6g
752,https://www.youtube.com/watch?v=O7609yfn88E
753,https://www.youtube.com/watch?v=UgdUPzGsWwI
754,https://www.youtube.com/watch?v=lf-KK2iWVkE
755,https://www.youtube.com/watch?v=yx_yQuQjOIo


Consumindo a api:

In [222]:
"""from googleapiclient.discovery import build
import pandas as pd

api_key = 'INSIRA SUA CHAVE API AQUI'

youtube = build('youtube', 'v3', developerKey=api_key)

def get_publish_date(video_url):
    try:
        # extract the video ID from the URL
        video_id = video_url.split('v=')[-1]

        # get the video details
        video_details = youtube.videos().list(part='snippet', id=video_id).execute()

        if video_details['items']:
            # get the publish date
            publish_date = video_details['items'][0]['snippet']['publishedAt']
            return publish_date
        else:
            print(f"Não foram encontradas informações  para o vídeo {video_url}")
            return None

    except Exception as e:
        print(f"Falha em conseguir a data de publicação para o vídeo {video_url}, error: {str(e)}")
        return None

df_urls['Publish Date'] = df_urls['YouTube URL'].apply(get_publish_date)"""

'from googleapiclient.discovery import build\nimport pandas as pd\n\napi_key = \'INSIRA SUA CHAVE API AQUI\'\n\nyoutube = build(\'youtube\', \'v3\', developerKey=api_key)\n\ndef get_publish_date(video_url):\n    try:\n        # extract the video ID from the URL\n        video_id = video_url.split(\'v=\')[-1]\n\n        # get the video details\n        video_details = youtube.videos().list(part=\'snippet\', id=video_id).execute()\n\n        if video_details[\'items\']:\n            # get the publish date\n            publish_date = video_details[\'items\'][0][\'snippet\'][\'publishedAt\']\n            return publish_date\n        else:\n            print(f"Não foram encontradas informações  para o vídeo {video_url}")\n            return None\n\n    except Exception as e:\n        print(f"Falha em conseguir a data de publicação para o vídeo {video_url}, error: {str(e)}")\n        return None\n\ndf_urls[\'Publish Date\'] = df_urls[\'YouTube URL\'].apply(get_publish_date)'

Vamos analisar essa url sem id. Antes, salvaremos o dataframe para não precisar consumir a `api` novamente caso o `kernel` precise ser reiniciado.

In [223]:
# df_urls.to_csv("../data/trusted/published_dates.csv", index=False)

In [224]:
df_urls = pd.read_csv("../data/trusted/published_dates.csv")

In [225]:
df_urls.info()
null_url = df_urls[df_urls.isnull().any(axis=1)].copy()
null_url

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YouTube URL   756 non-null    object
 1   Publish Date  755 non-null    object
dtypes: object(2)
memory usage: 11.9+ KB


,YouTube URL,Publish Date
68,https://www.youtube.com/watch?v=,NaN


Analisaremos os nulos no df antigo, que possui as demais informações.

In [226]:
empty_id = weekly_charts[weekly_charts["YouTube URL"].str.len() < len("https://www.youtube.com/watch?v=")+1]
empty_id = empty_id.reset_index()
empty_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           80 non-null     int64         
 1   Rank            80 non-null     int64         
 2   Previous Rank   80 non-null     int64         
 3   Track Name      80 non-null     object        
 4   Artist Names    80 non-null     object        
 5   Weeks on Chart  80 non-null     int64         
 6   Views           80 non-null     int64         
 7   Weekly Growth   80 non-null     float64       
 8   YouTube URL     80 non-null     object        
 9   week_ref        80 non-null     int64         
 10  week_open       80 non-null     datetime64[ns]
 11  week_close      80 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(6), object(3)
memory usage: 7.6+ KB


In [227]:
track_names = empty_id["Track Name"].unique()
track_names

array(['Ela me Falou Que Quer Rave', 'Bagulho Louco',
       'Olho no Olho Coraçao Acelera', 'Set Wesley Alemão',
       'Lovezinho (Sexy Version) (Remix)', 'Duas',
       'Cadê Seu Namorado Moça?'], dtype=object)

Vendo como se espalham pelas semanas para saber se é um problema de uma das páginas específicas ou algo da api do youtube.

In [228]:
empty_id["week_ref"].unique()

array([70, 69, 68, 67, 66, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47,
       46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30,
       29, 28, 27, 26, 25, 24, 23, 22, 21, 19, 17, 16, 15, 14, 13, 12, 11,
       10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0], dtype=int64)

É algo geral. Verificando nas páginas, percebo que algumas poucas vezes o vídeo fica sem link.

In [229]:
print(f"Quantidade de linhas sem id: {len(empty_id)}")
print(f"Quantidade de faixas sem id: {len(track_names)}")

Quantidade de linhas sem id: 80
Quantidade de faixas sem id: 7


Por desencargo, procurando se há urls com id semi preenchido. Teria apitado no `try-except` na hora de executar o `for` para consumir a `api`, mas nunca se sabe.

In [230]:
test = weekly_charts[weekly_charts["YouTube URL"].str.len() < len("https://www.youtube.com/watch?v=yx_yQuQjOIo")]
len(test)

80

In [231]:
tracks_to_total = len(track_names)/len(df_urls)
events_to_total = len(empty_id)/len(weekly_charts)

print(f"As {len(track_names)} músicas representam {tracks_to_total*100:.2f}% do total de faixas")
print(f"As {len(empty_id)} linhas representam {events_to_total*100:.2f}% do total de eventos no dataframe")

As 7 músicas representam 0.93% do total de faixas
As 80 linhas representam 1.13% do total de eventos no dataframe


Mesmo número que o anterior, são só aqueles mesmo.

Temos 80 linhas sem id, num total, correspondendo a 7 músicas.

Tais números representam amostra reduzida to dataframe original.

De qualquer forma, isso seria uma decisão a ser tomada em conjunto com a equipe, para saber quais os métodos e técnicas adequadas para proceder nesses casos.

Para fins desse exercício, buscando deixar o processo fluido e replicar um ambiente semi-automarizado, essas linhas serão `dropadas` dos dataframes e excluídas da análise.

In [232]:
mask_drop_empities = weekly_charts["YouTube URL"].str.len() < 33
weekly_charts = weekly_charts[~mask_drop_empities]

In [233]:
weekly_charts.reset_index(drop = True, inplace=True)
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7020 non-null   int64         
 1   Previous Rank   7020 non-null   int64         
 2   Track Name      7020 non-null   object        
 3   Artist Names    7020 non-null   object        
 4   Weeks on Chart  7020 non-null   int64         
 5   Views           7020 non-null   int64         
 6   Weekly Growth   7020 non-null   float64       
 7   YouTube URL     7020 non-null   object        
 8   week_ref        7020 non-null   int64         
 9   week_open       7020 non-null   datetime64[ns]
 10  week_close      7020 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 603.4+ KB


In [234]:
mask_drop_empities = df_urls["YouTube URL"].str.len() < 33
df_urls = df_urls[~mask_drop_empities]
df_urls.reset_index(drop=True, inplace = True)
df_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YouTube URL   755 non-null    object
 1   Publish Date  755 non-null    object
dtypes: object(2)
memory usage: 11.9+ KB


# Juntando os dataframes (`join`)

In [235]:
df_urls.info()
weekly_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YouTube URL   755 non-null    object
 1   Publish Date  755 non-null    object
dtypes: object(2)
memory usage: 11.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7020 entries, 0 to 7019
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7020 non-null   int64         
 1   Previous Rank   7020 non-null   int64         
 2   Track Name      7020 non-null   object        
 3   Artist Names    7020 non-null   object        
 4   Weeks on Chart  7020 non-null   int64         
 5   Views           7020 non-null   int64         
 6   Weekly Growth   7020 non-null   float64       
 7   YouTube URL     7020 non-null   object        
 8   week_ref        7020 non-null   int64         
 9   wee

Será utilizado o método .merge() do pandas para realizar um `inner join` e será verificado se tudo ocorreu como esperado

In [236]:
refined_df = weekly_charts.merge(df_urls, on= "YouTube URL")

In [237]:
len(refined_df) == len(weekly_charts)

True

In [238]:
count_strings = sum(refined_df["YouTube URL"].str.len() < 33)
count_strings

0

Para finalizar este notebook, convertemos a coluna publised dates para datas e salvaremos os arquivos como refined, isto é, arquivos prontos para análise

In [239]:
refined_df.head()

,Rank,Previous Rank,Track Name,Artist Names,Weeks on Chart,Views,Weekly Growth,YouTube URL,week_ref,week_open,week_close,Publish Date
0,1,1,Malvada,Zé Felipe,2,21185310,0.098,https://www.youtube.com/watch?v=r0mNwyywHIY,70,2022-02-04,2022-02-10,2022-01-28T15:00:12Z
1,1,1,Malvada,Zé Felipe,3,17507479,-0.174,https://www.youtube.com/watch?v=r0mNwyywHIY,69,2022-02-11,2022-02-17,2022-01-28T15:00:12Z
2,1,1,Malvada,Zé Felipe,4,16148340,-0.078,https://www.youtube.com/watch?v=r0mNwyywHIY,68,2022-02-18,2022-02-24,2022-01-28T15:00:12Z
3,1,1,Malvada,Zé Felipe,5,17136822,0.061,https://www.youtube.com/watch?v=r0mNwyywHIY,67,2022-02-25,2022-03-03,2022-01-28T15:00:12Z
4,1,1,Malvada,Zé Felipe,6,14849090,-0.133,https://www.youtube.com/watch?v=r0mNwyywHIY,66,2022-03-04,2022-03-10,2022-01-28T15:00:12Z


In [240]:
refined_df["Publish Date"] = pd.to_datetime(refined_df["Publish Date"]).dt.date
refined_df["Publish Date"] = pd.to_datetime(refined_df["Publish Date"])


In [241]:
refined_df.info()
refined_df["Publish Date"].head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7020 entries, 0 to 7019
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7020 non-null   int64         
 1   Previous Rank   7020 non-null   int64         
 2   Track Name      7020 non-null   object        
 3   Artist Names    7020 non-null   object        
 4   Weeks on Chart  7020 non-null   int64         
 5   Views           7020 non-null   int64         
 6   Weekly Growth   7020 non-null   float64       
 7   YouTube URL     7020 non-null   object        
 8   week_ref        7020 non-null   int64         
 9   week_open       7020 non-null   datetime64[ns]
 10  week_close      7020 non-null   datetime64[ns]
 11  Publish Date    7020 non-null   datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(5), object(3)
memory usage: 713.0+ KB


0   2022-01-28
1   2022-01-28
2   2022-01-28
3   2022-01-28
4   2022-01-28
Name: Publish Date, dtype: datetime64[ns]

Finalmente, adicionaremos a coluna `frontline`, que receberá valores booleanos `0` e `1`. A depender do workflow, aqui também poderiam ser recebidos os valores `True` e `False`, geralmente aceitos pela maioria dos softwares de hoje em dia.

In [242]:

refined_df['frontline'] = (refined_df['week_open'] - refined_df['Publish Date']).dt.days <= 42*30


Checando com operadores `booleanos`

In [243]:
# Checking that all 'frontline' tracks were published within the last 42 months
frontline_check = all((refined_df[refined_df['frontline']]['week_open'] - refined_df[refined_df['frontline']]['Publish Date']).dt.days <= 1260)

# Checking that all non-'frontline' tracks were published more than 42 months ago
non_frontline_check = all((refined_df[~refined_df['frontline']]['week_open'] - refined_df[~refined_df['frontline']]['Publish Date']).dt.days > 1260)

print("Todos os valores `frontline` possuem data de lançamento abaixo de 42 meses atrás:", frontline_check)
print("Todos os valores `não-frontline` (catálogos) possuem data de lançamento  acima de 42 meses atrás:", non_frontline_check)


Todos os valores `frontline` possuem data de lançamento abaixo de 42 meses atrás: True
Todos os valores `não-frontline` (catálogos) possuem data de lançamento  acima de 42 meses atrás: True


In [244]:
refined_df["frontline"].value_counts()


True     6789
False     231
Name: frontline, dtype: int64

In [245]:
refined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7020 entries, 0 to 7019
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rank            7020 non-null   int64         
 1   Previous Rank   7020 non-null   int64         
 2   Track Name      7020 non-null   object        
 3   Artist Names    7020 non-null   object        
 4   Weeks on Chart  7020 non-null   int64         
 5   Views           7020 non-null   int64         
 6   Weekly Growth   7020 non-null   float64       
 7   YouTube URL     7020 non-null   object        
 8   week_ref        7020 non-null   int64         
 9   week_open       7020 non-null   datetime64[ns]
 10  week_close      7020 non-null   datetime64[ns]
 11  Publish Date    7020 non-null   datetime64[ns]
 12  frontline       7020 non-null   bool          
dtypes: bool(1), datetime64[ns](3), float64(1), int64(5), object(3)
memory usage: 719.8+ KB


Salvando

In [246]:
refined_df.to_pickle("../data/refined/refined.pkl")
refined_df.to_csv("../data/refined/refined.csv", index=False)